# Neuroscience using `fastplotlib` :D

This notebook will build up a complex visualization using `fastplotlib`, in conjunction with other Python neuroscience packages, to exemplify how `fastplotlib` can be a powerful tool in analysis and visualization of neural data!

In [36]:
import fastplotlib as fpl
import pynapple as nap
import numpy as np
from pathlib import Path
from ipywidgets.widgets import VBox, HBox, IntSlider
from mesmerize_core import *

### Opening behavior videos using **Lazy Loading** and visualizing them using `fastplotlib`.

#### **Lazy Loading**: strategy to conserve RAM by dynamically loading in frames for visualization only as needed 

Behavioral and neural data collected during experiments can be up to **terabytes** in size making it impossible to load all of the data into RAM at once for visualization and analysis. ***Lazy Loading*** allows us to bypass this memory space constraint without overburdening our resources!

In [2]:
# helper class from mesmerize_core for lazy loading
from mesmerize_core.arrays import LazyVideo

In [14]:
data = Path('/data/kushal/cortex-learning/2p-trial-exps/eaf1/behavior/session4/')

#### Get video paths for a single trial

In [17]:
trial_one = sorted(list(data.glob("*018.avi")))
trial_one

[PosixPath('/data/kushal/cortex-learning/2p-trial-exps/eaf1/behavior/session4/eaf1-s4_front_v018.avi'),
 PosixPath('/data/kushal/cortex-learning/2p-trial-exps/eaf1/behavior/session4/eaf1-s4_side_v018.avi')]

#### Make an `ImageWidget` to view the trial

In [18]:
iw = fpl.ImageWidget(
    data=[LazyVideo(v) for v in trial_one], 
    names=["front", "side"],
    histogram_widget=False)

iw.show()

/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel

RFBOutputContext()

/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/traitlets/traitlets.py:1387: DeprecationWarning: Passing unrecognized arguments to super(Button).__init__(value=False).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

### Visualizing results of running `CaImAn` via `mesmerize-core` on calcium imaging data.

In [6]:
set_parent_raw_data_path('/data/kushal/cortex-learning/2p-trial-exps/')

PosixPath('/data/kushal/cortex-learning/2p-trial-exps')

In [7]:
# load mesmerize batch that has already been run for this session
df = load_batch(get_parent_raw_data_path().joinpath('eaf1/calcium/batch/batch.pickle'))
df

,algo,item_name,input_movie_path,params,outputs,comments,uuid,added_time,ran_time,algo_duration
0,mcorr,eaf1_session1,eaf1/calcium/session1/concat_session1.tif,"{'main': {'max_shifts': (24, 24), 'strides': (...",{'mean-projection-path': 90cc9605-3fc0-4c4c-8c...,None,90cc9605-3fc0-4c4c-8c61-5c6db19ac1f8,None,None,None
1,mcorr,eaf1_session2,eaf1/calcium/session2/concat_session2.tif,"{'main': {'max_shifts': (24, 24), 'strides': (...",{'mean-projection-path': 2ae18c23-08de-4987-b5...,None,2ae18c23-08de-4987-b5e4-f805c196fc4d,None,None,None
2,mcorr,eaf1_session3,eaf1/calcium/session3/concat_session3.tif,"{'main': {'max_shifts': (24, 24), 'strides': (...",{'mean-projection-path': 3efacee7-1453-40ef-87...,None,3efacee7-1453-40ef-87ed-acf6369fee85,None,None,None
3,mcorr,eaf1_session4,eaf1/calcium/session4/concat_session4.tif,"{'main': {'max_shifts': (24, 24), 'strides': (...",{'mean-projection-path': f9325d96-3ef7-4295-88...,None,f9325d96-3ef7-4295-8864-6858e7d54306,None,None,None
4,mcorr,eaf1_session5,eaf1/calcium/session5/concat_session5.tif,"{'main': {'max_shifts': (12, 12), 'strides': (...",{'mean-projection-path': 5e622b7f-0985-413e-b9...,None,5e622b7f-0985-413e-b9db-51970e8a655e,None,None,None
5,mcorr,eaf1_session6,eaf1/calcium/session6/concat_session6.tif,"{'main': {'max_shifts': (12, 12), 'strides': (...",{'mean-projection-path': 0469b0e4-75e8-4af0-97...,None,0469b0e4-75e8-4af0-97cf-bde0550ce109,None,None,None
6,cnmf,eaf1_session4,f9325d96-3ef7-4295-8864-6858e7d54306/f9325d96-...,"{'main': {'fr': 15, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': 6f6255f2-b8fc-49ba-bc...,None,6f6255f2-b8fc-49ba-bccd-0e10cea98819,None,None,None
7,cnmf,eaf1_session5,5e622b7f-0985-413e-b9db-51970e8a655e/5e622b7f-...,"{'main': {'fr': 15, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': 9e31ce67-883f-43fe-b7...,None,9e31ce67-883f-43fe-b78b-2fc95bd452b3,None,None,None
8,cnmf,eaf1_session6,0469b0e4-75e8-4af0-97cf-bde0550ce109/0469b0e4-...,"{'main': {'fr': 15, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': 4cae2d6e-1ea5-47e0-8a...,None,4cae2d6e-1ea5-47e0-8abf-e5d282fb64c1,None,None,None
9,cnmf,eaf1_session5,5e622b7f-0985-413e-b9db-51970e8a655e/5e622b7f-...,"{'main': {'fr': 15, 'p': 1, 'nb': 2, 'merge_th...",{'mean-projection-path': a1f58223-978b-42bc-a1...,None,a1f58223-978b-42bc-a173-d31c0327e469,None,None,None


#### Get the motion corrected and reconstructed movie

In [8]:
rcm = df.iloc[26].cnmf.get_rcm()
raw = df.iloc[26].caiman.get_input_movie()

In [9]:
neural_iw = fpl.ImageWidget(
    data=[raw, rcm],
    names=["raw", "rcm"]
)

neural_iw.show(sidecar=True)

/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel

RFBOutputContext()

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/traitlets/traitlets.py:1387: DeprecationWarning: Passing unrecognized arguments to super(Button).__init__(value=False).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `co

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='auto-scale scene'), Button(icon='align-center', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='auto-center scene'), ToggleButton(value=True, icon='hand-pointer', layout=Layout(width='auto'), tooltip='panzoom controller'), ToggleButton(value=True, description='1:1', layout=Layout(width='auto'), style=ToggleButtonStyle(font_weight='bold'), tooltip='maintain aspect'), Button(icon='arrow-down', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='y-axis direction'), Button(icon='draw-polygon', layout=Layout(width='auto'), style=ButtonStyle(), tooltip='add PolygonSelector'), ToggleButton(value=False, icon='video', layout=Layout(width='auto'), tooltip='record'), Dropdown(description='Subplots:', layout=Layout(width='200px'), options=('raw', 'rcm'), value='raw'))), IpywidgetImageWidgetToolbar(children=(HBox

#### Add neural contours to reconstructed movie

In [10]:
# get the contours and center of masses using mesmerize_core
contours, coms = df.iloc[26].cnmf.get_contours(component_indices="good", swap_dim=False)

# add the contours to the rcm plot
contours_graphic = neural_iw.gridplot["rcm"].add_line_collection(
                                                    data=contours, 
                                                    colors="red",
                                                    thickness=2,
                                                    name="contours")

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")


### Creating an interactive viualization to help with analysis

Will need a euclidean helper function to indentify which contours has been clicked on. We hope to soon include this, and other common callback functions, in our interactivity system :D

In [11]:
def euclidean(source, target, event, new_data):
    """maps click events to contour"""
    # calculate coms of line collection
    indices = np.array(event.pick_info["index"])
    
    coms = list()

    for contour in target.graphics:
        coors = contour.data()[~np.isnan(contour.data()).any(axis=1)]
        com = coors.mean(axis=0)
        coms.append(com)

    # euclidean distance to find closest index of com 
    indices = np.append(indices, [0])
    
    ix = int(np.linalg.norm((coms - indices), axis=1).argsort()[0])
    
    target.set_feature(feature="colors", new_data=new_data, indices=ix)
    
    return None

#### Create a plot to display the associated temporal data for each identified neuron

In [12]:
# get temporal components
temporal = df.iloc[26].cnmf.get_temporal(component_indices="good")

# temporal plot
plot_temporal = fpl.Plot(size=(600,100))
plot_temporal.add_line(temporal[0], colors="magenta")

# show temporal plot and mcorr/rcm plot in ipywidgets VBox 
VBox([neural_iw.show(), plot_temporal.show()])

/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel

RFBOutputContext()

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:34: UserWarning: converting float64 array to float32
  warn(f"converting {array.dtype} array to float32")
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/traitlets/traitlets.py:1387: DeprecationWarning: Passing unrecognized arguments to super(Button).__init__(value=False).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `co

#### Interacitvity of `Graphics` using `link()` and callback functions

In [13]:
# link image to contours
image_graphic = neural_iw.gridplot[0,1]["image_widget_managed"]

image_graphic.link(
    "click",
    target=contours_graphic,
    feature="colors", 
    new_data="cyan", 
    callback=euclidean
)

# a vertical line that is syncronized to the image widget "t" (timepoint) slider
def update_timepoint(ev):
    ix = ev.pick_info["selected_index"]
    neural_iw.sliders["t"].value = ix


# callback function to display correct temporal trace
def generate_temporal(ev):
    # clear the plot 
    plot_temporal.clear()
    
    # get data of selected ix
    data = temporal[ev.pick_info["collection-index"]]
    
    # add trace to plot 
    temporal_graphic = plot_temporal.add_line(data, colors="magenta")
    
    # add selector synced to "t" slider
    selector = temporal_graphic.add_linear_selector()
    selector.selection.add_event_handler(update_timepoint)
    
    plot_temporal.auto_scale()

# add event handler so that temporal trace is generated when contour is clicked on
contours_graphic[:].colors.add_event_handler(generate_temporal)

# thickness of contour
contours_graphic.link("colors", target=contours_graphic, feature="thickness", new_data=5)

### Syncing behavior and calcium data using `pynapple`

In [20]:
import tifffile

In [22]:
concat_behavior = tifffile.memmap("/home/clewis7/Desktop/sfn_data/preconcat.tiff")

#### Recording Frame Rates:

In [24]:
behavior_fr = 500
calcium_fr = 15.2414

In [26]:
concat_behavior.shape

(318850, 256, 336)

#### Create `pynapple` tensors for behavior and calcium data based on frame rate

In [57]:
t_behavior = np.linspace(0, concat_behavior.shape[0] / behavior_fr, concat_behavior.shape[0])

In [58]:
tsd_video = nap.TsdTensor(t_behavior, concat_behavior)

In [59]:
t_calcium = np.linspace(0, raw.shape[0] / calcium_fr, raw.shape[0])

In [60]:
tsd_calcium = nap.TsdTensor(t_calcium, raw)

#### Create plot for concatenated behavior videos

In [61]:
behavior_plot = fpl.Plot()

behavior_plot.add_image(tsd_video[0], cmap="gray")

/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel

RFBOutputContext()

<weakproxy at 0x7f00bc759300 to ImageGraphic at 0x7f00cc4680d0>

#### Create slider that updates behavior and calcium data so they are aligned

In [65]:
synced_time = IntSlider(min=0, max=19 * 1_000)

def update_time(change):
    time_ms = change["new"]
    frame_raw = tsd_calcium.get(time_ms, time_ms + 67, time_units="ms")
    neural_iw.gridplot["raw"].graphics[0].data = frame_raw
    behavior_plot.graphics[0].data = tsd_video.get(time_ms, time_ms + 2, time_units="ms").values

synced_time.observe(update_time, "value")

/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


In [66]:
VBox([HBox([neural_iw.show(), behavior_plot.show()]), synced_time])

/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/traitlets/traitlets.py:1387: DeprecationWarning: Passing unrecognized arguments to super(Button).__init__(value=False).
object.__init__() takes exactly one argument (the instance to initialize)
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets/widgets/widget.py:469: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)
/home/clewis7/venvs/fpl/lib/python3.11/site-packages/ipywidgets